### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 43.871332
2 		 15.156243
3 		 3.973459
4 		 2.743113
5 		 2.646949
6 		 2.607239
7 		 2.570704
8 		 2.531287
9 		 2.487120
10 		 2.437153


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.91958314   8.97010269  10.27758185  10.4836529 ]
 [  8.68316005   7.41973787   8.96331669   8.89910642]
 [  8.07629428   6.97825595   8.2927928    7.5579642 ]
 [  9.69204688   8.51925442   9.8073088    9.56918404]
 [  8.78410498   7.34997792   8.92840862   8.46980286]
 [  6.73100169   6.14823868   7.43020919   6.65377353]
 [  9.16742083   7.72944637   9.93460985   9.16224908]
 [  8.56605162   6.94473085   8.62821546   8.09064511]
 [  8.28340111   7.58401086   8.82574233   8.58003143]
 [  7.74698095   6.329846     8.30910317   7.63104636]
 [  9.07095845   7.87590062   9.27131001   8.72155395]
 [  9.69710335   8.55567398   9.59732144   9.81991177]
 [  9.31143169   8.09804394   9.86780138   9.09090504]
 [  6.58933391   5.88103322   6.75450855   6.77802252]
 [  9.76775134   8.5682495   10.05867951   9.84872701]
 [  9.35615753   8.47692184  10.32478784   9.91627102]
 [  8.43150157   7.19923498   8.9154626    8.04165589]
 [  7.21458448   6.5971611    6.9973185    7.2010172 ]
 [  8.1594

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 40.006167
2 		 13.085585
3 		 3.855223
4 		 2.831494
5 		 2.726456
6 		 2.691329
7 		 2.666142
8 		 2.641662
9 		 2.615530
10 		 2.586843
11 		 2.554994
12 		 2.519424
13 		 2.479571
14 		 2.434882
15 		 2.384828
16 		 2.328936
17 		 2.266835
18 		 2.198297
19 		 2.123305
20 		 2.042102
21 		 1.955243
22 		 1.863627
23 		 1.768491
24 		 1.671368
25 		 1.573984
26 		 1.478130
27 		 1.385497
28 		 1.297519
29 		 1.215248
30 		 1.139286
31 		 1.069779
32 		 1.006467
33 		 0.948784
34 		 0.895966
35 		 0.847151
36 		 0.801477
37 		 0.758135
38 		 0.716421
39 		 0.675753
40 		 0.635683
41 		 0.595898
42 		 0.556219
43 		 0.516593
44 		 0.477079
45 		 0.437840
46 		 0.399121
47 		 0.361226
48 		 0.324496
49 		 0.289281
50 		 0.255912
51 		 0.224680
52 		 0.195813
53 		 0.169464
54 		 0.145708
55 		 0.124544
56 		 0.105898
57 		 0.089640
58 		 0.075598
59 		 0.063571
60 		 0.053345
61 		 0.044703
62 		 0.037436
63 		 0.031350
64 		

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.0000004   10.00000039  10.00000018   9.99999942]
 [ 10.00000103   4.00000094   9.00000042   9.99999858]
 [  8.00000029   9.00000029  10.00000012   4.99999957]
 [  9.00000042   8.00000028  10.00000012   9.99999947]
 [  9.99999921   4.99999929   8.99999965   9.00000104]
 [  6.00000075   4.00000062  10.00000028   5.99999901]
 [  8.99999939   7.99999951   9.99999975   9.00000075]
 [ 10.00000066   5.00000055   9.00000024   7.99999912]
 [  7.00000082   8.0000006   10.00000027   7.99999897]
 [  8.99999948   4.99999955   8.99999977   7.00000066]
 [  9.00000004   8.00000008  10.00000001   7.99999988]
 [  9.00000023  10.00000024  10.00000008   8.99999963]
 [  9.99999923   8.99999932   9.99999967   8.00000101]
 [  5.00000037   8.00000031   5.00000013   7.9999995 ]
 [  9.99999979   7.99999978   9.99999989  10.00000028]
 [  8.99999974   8.99999973   9.99999988  10.00000036]
 [  9.00000126   8.00000105   8.00000048   7.99999834]
 [  9.99999978   7.99999981   0.9999999   10.00000027]
 [  4.9999

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.019655
2 		 14.474907
3 		 4.291064
4 		 2.992446
5 		 2.808754
6 		 2.744458
7 		 2.703438
8 		 2.665909
9 		 2.626218
10 		 2.582239
11 		 2.532847
12 		 2.477315
13 		 2.415205
14 		 2.346402
15 		 2.271174
16 		 2.190230
17 		 2.104747
18 		 2.016340
19 		 1.926956
20 		 1.838708
21 		 1.753651
22 		 1.673552
23 		 1.599699
24 		 1.532786
25 		 1.472904
26 		 1.419613
27 		 1.372081
28 		 1.329250
29 		 1.289983
30 		 1.253187
31 		 1.217891
32 		 1.183293
33 		 1.148775
34 		 1.113909
35 		 1.078445
36 		 1.042297
37 		 1.005523
38 		 0.968312
39 		 0.930953
40 		 0.893814
41 		 0.857308
42 		 0.821867
43 		 0.787905
44 		 0.755790
45 		 0.725824
46 		 0.698222
47 		 0.673108
48 		 0.650517
49 		 0.630396
50 		 0.612630
51 		 0.597048
52 		 0.583445
53 		 0.571601
54 		 0.561288
55 		 0.552286
56 		 0.544389
57 		 0.537408
58 		 0.531173
59 		 0.525535
60 		 0.520365
61 		 0.515548
62 		 0.510986
63 		 0.506595
64 		

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.950110641624823:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.249346
2 		 10.725681
3 		 7.355252
4 		 5.678395
5 		 4.624682
6 		 3.887137
7 		 3.337050
8 		 2.909603
9 		 2.567903
10 		 2.289030
11 		 2.057889
12 		 1.864114
13 		 1.700308
14 		 1.560968
15 		 1.441835
16 		 1.339509
17 		 1.251220
18 		 1.174692
19 		 1.108048
20 		 1.049735


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [15]:
np.count_nonzero(~np.isnan(first_1000_users))

10852

In [17]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0] * first_1000_users.shape[1] - num_ratings # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [18]:
print(np.dot(user_mat, movie_mat))

[[ 4.49126546  4.10921876  1.32724791 ...,  2.37143803  1.41638435
   4.64640988]
 [ 4.28519323  3.57804466  1.52920443 ...,  2.55916387  1.4288622
   4.05739398]
 [ 3.75851656  3.33333636  1.09321379 ...,  1.84836563  1.02643142
   4.06506277]
 ..., 
 [ 3.79773776  3.08748434  1.32148664 ...,  1.87356932  0.95892556
   3.8327494 ]
 [ 5.15334018  4.10733977  1.83517799 ...,  2.86031026  1.45825491
   5.12694824]
 [ 3.45480736  2.97210147  1.14057355 ...,  1.95910285  1.10328228
   3.43735204]]


In [19]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
